In [50]:
import streamlit as st
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Content, FinishReason, SafetySetting
import vertexai.preview.generative_models as generative_models
import os
import requests
import google.auth.transport.requests
from google.oauth2 import service_account
import google.oauth2.id_token
import vertexai
import re

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\ASUS\Documents\Job\Datalabs\2025\service_account\dla-gen-ai-specialization-a03e18685e07.json" # Isi dengan Service Account

In [12]:
PROJECT_ID = "844021890758"
COLLECTION_NAME = "default_collection"
ENGINE_NAME = "demo2-test_1736323106527"

In [4]:
def search_with_answer(query,session,agent):
    url_search = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/servingConfigs/default_search:search"
    # API credential
    credentials = service_account.Credentials.from_service_account_file(
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'],
        scopes=['https://www.googleapis.com/auth/cloud-platform']
    )
    request_token = google.auth.transport.requests.Request()
    credentials.refresh(request_token)
    headers = {
        "Authorization": f"Bearer {credentials.token}",
        "Content-Type": "application/json"
    }
    # ngelakuin search file dulu baru summary, ini buat data searchnya
    if session == "":
        session = f"projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/sessions/-"
    else:
        pass
    request_data_search = {
        "query": query,
        "pageSize": 10,
        "queryExpansionSpec": {
            "condition": "AUTO"
        },
        "spellCorrectionSpec": {
            "mode": "AUTO"
        },
        "contentSearchSpec": {
            "snippetSpec": {
                "returnSnippet": True
            },
            "extractiveContentSpec": {
                "maxExtractiveAnswerCount": 1
            }
        },
        "session": session
    }
    request_search = requests.post(url=url_search, json=request_data_search,headers=headers)
    search = request_search.json()
    # ini bagian api call untuk summary
    url_answer = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/servingConfigs/default_search:answer"

    request_data_answer = {
        "query": {
            "text": query,
            "queryId": search['sessionInfo']['queryId']
        },
        "session": search['sessionInfo']['name'],
        "relatedQuestionsSpec": {
            "enable": True
        },
        "answerGenerationSpec": {
            "ignoreAdversarialQuery": True,
            "ignoreNonAnswerSeekingQuery": True,
            "ignoreLowRelevantContent": True,
            "includeCitations": True,
            "promptSpec": {
                "preamble": """Anda adalah agent chatbot internal untuk perusahaan WOM Finance Jawablah dalam bahasa indonesia jangan dengan bahasa inggris,untuk jawaban yang tidak ditemukan harap balas dengan "Jawaban tidak ditemukan, silahkan menghubungi tim OPP pada email OPP@wom.co.id. Terima Kasih" """
            },
            "modelSpec": {
                "modelVersion": "gemini-1.5-flash-001/answer_gen/v2"
            }
        }
    }

    request_answer = requests.post(url_answer, json=request_data_answer,headers=headers)
    answer = request_answer.json()

    return answer

In [24]:
query = "laptop reccomendations"
session = ''
agent = "demo2-test_1736323106527"

In [25]:
url_search = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/servingConfigs/default_search:search"
# API credential
credentials = service_account.Credentials.from_service_account_file(
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'],
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)
request_token = google.auth.transport.requests.Request()
credentials.refresh(request_token)
headers = {
    "Authorization": f"Bearer {credentials.token}",
    "Content-Type": "application/json"
}
# ngelakuin search file dulu baru summary, ini buat data searchnya
if session == "":
    session = f"projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/sessions/-"
else:
    pass
request_data_search = {
    "query": query,
    "pageSize": 10,
    "queryExpansionSpec": {
        "condition": "AUTO"
    },
    "spellCorrectionSpec": {
        "mode": "AUTO"
    },
    "contentSearchSpec": {
        "snippetSpec": {
            "returnSnippet": True
        },
        "extractiveContentSpec": {
            "maxExtractiveAnswerCount": 1
        }
    },
    "session": session
}
request_search = requests.post(url=url_search, json=request_data_search,headers=headers)
search = request_search.json()

In [26]:
# ini bagian api call untuk summary
url_answer = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_ID}/locations/global/collections/{COLLECTION_NAME}/engines/{agent}/servingConfigs/default_search:answer"

request_data_answer = {
    "query": {
        "text": query,
        "queryId": search['sessionInfo']['queryId']
    },
    "session": search['sessionInfo']['name'],
    "relatedQuestionsSpec": {
        "enable": True
    },
    "answerGenerationSpec": {
        "ignoreAdversarialQuery": True,
        "ignoreNonAnswerSeekingQuery": True,
        "ignoreLowRelevantContent": True,
        "includeCitations": True,
        "promptSpec": {
            "preamble": """"""
        },
        "modelSpec": {
            "modelVersion": "gemini-1.5-flash-001/answer_gen/v2"
        }
    }
}

request_answer = requests.post(url_answer, json=request_data_answer,headers=headers)
answer = request_answer.json()

In [27]:
answer['answer']['answerText']

'A summary could not be generated for your search query. Here are some search results.'

In [39]:
search['results'][0]['document']['derivedStructData']['link']

'gs://gen-ai-specialization-demo2/data-hugging-face/text/output_7.txt'

In [49]:
reference = ''
sources_link = []
for i in range(len(search['results'])):
    reference +=f"[{i}]{search['results'][i]['document']['derivedStructData']['extractive_answers'][0]['content']}\n"
    gcs_uri = search['results'][i]['document']['derivedStructData']['link']
    # Extract data source
    source_exact = gcs_uri.replace("gs://","https://storage.cloud.google.com/")
    source_exact = source_exact.replace(" ","%20")
    sources_link.append(f"[{i}]{source_exact} \n\n")
sources_link_join = ''.join(sources_link)

In [52]:
import re

def check_numbered_sequence(text, limit):
    """
    Checks if a text contains a numbered sequence like '[1]', '[2]', '[3]' up to a certain limit.

    Args:
        text: The input text string.
        limit: The upper limit of the numbered sequence.

    Returns:
        True if the sequence is found up to the limit, False otherwise.
    """
    for i in range(1, limit + 1):
        expected_pattern = r"\[" + str(i) + r"\]"
        if not re.search(expected_pattern, text):
            return False
    return True

# Example usage:
text1 = "This is a text with [1], [2], [3], [4], and [5]."
limit1 = 5
print(f"Text 1: {check_numbered_sequence(text1, limit1)}")

text2 = "This text has [1], [2], and [4], but is missing [3]."
limit2 = 4
print(f"Text 2: {check_numbered_sequence(text2, limit2)}")

text3 = "This text has [1], [2], [3]."
limit3 = 3
print(f"Text 3: {check_numbered_sequence(text3, limit3)}")

text4 = "This text has [1] and [2]."
limit4 = 3
print(f"Text 4: {check_numbered_sequence(text4, limit4)}")

Text 1: True
Text 2: True
Text 3: True
Text 4: False
